In [ ]:
from src import *

# pip install email_notify
import email_notify

In [ ]:
today

In [ ]:
with RaspiLED() as led:
    led.check()

In [ ]:
############
# Freq Est #
############

sampling_rate = 20 #Hz

# freq_list = np.linspace(0.1, 0.4, 61)
# pwm_duty = None

freq_list = [0.2]
pwm_duty = np.arange(0, 101, 2).astype(int)
# pwm_duty = [1,3,5,7,9]

A = 5
samples = 50
repeat = 200

interval = 1/sampling_rate # s
exposure_time = 500e-6 # s, 500us
timeout_milisec = int(1.5*samples*interval*1e3) # ms

measurement = 'DI'

In [ ]:
def task(dcam: EasyDcam, alp: EasyALP4, ground_truth, led: RaspiLED=None, pwm_duty=0):
    print(f'Current sensor temperature is {dcam.ez_temperature()}')
    if dcam.ez_temperature() >= -30:
        raise RuntimeError("qCMOS's temperature is too high.")
    
    if led is not None:
        led.turn_on(int(pwm_duty))

    ground_truth = np.round(ground_truth, 5)
    pic_time = int(1 / (sampling_rate * ground_truth) / 2 * 1e6) # us

    alp.ez_load_seq([alp.ez_single_pixel(0), alp.ez_single_pixel(A)], pic_time)
    dcam.ez_exposure_time(exposure_time)
    dcam.ez_triggersource_masterpluse(samples, interval)

    if measurement.upper() == 'SPADE':
        dcam.ez_roi(**SPADE.ROI)
    elif measurement.upper() == 'DI':
        dcam.ez_roi(**DI.ROI)

    raw, timestamp = [], []
    for _ in tqdm(range(repeat)):
        dcam.buf_alloc(samples)
        dcam.cap_snapshot()

        alp.Run()
        sleep(1e-6)
        dcam.cap_firetrigger()

        dcam.ez_wait_capture(timeout_milisec)

        dcam.cap_stop()
        alp.Halt()

        raw_, timestamp_ = [], []
        for frame in range(samples):
            framedata_ = dcam.ez_read_buf(frame)
            raw_.append(framedata_[0])
            timestamp_.append(framedata_[1])

        dcam.buf_release()

        raw.append(raw_)
        timestamp.append(timestamp_)

    raw = np.array(raw)
    timestamp = np.array(timestamp)

    if not os.path.exists(f'__raw__/{today}/{measurement.lower()}_{A}px'):
        os.makedirs(f'__raw__/{today}/{measurement.lower()}_{A}px')

    np.save(f'__raw__/{today}/{measurement.lower()}_{A}px/{measurement.lower()}_{A}px_f{ground_truth}_d{pwm_duty}_raw.npy', raw)
    np.save(f'__raw__/{today}/{measurement.lower()}_{A}px/{measurement.lower()}_{A}px_f{ground_truth}_d{pwm_duty}_timestamp.npy', timestamp)



@email_notify.decorator('hcnzj@qq.com')
def main():
    if pwm_duty is None:
        with EasyDcam() as dcam, EasyALP4() as alp:
            for index, f in enumerate(freq_list):
                print(f'({index}): f{np.round(f, 5)}_d0', end=' ')
                task(dcam, alp, f)
    else: 
        with RaspiLED() as led, EasyDcam() as dcam, EasyALP4() as alp:
            for f in freq_list:
                for index, d in enumerate(pwm_duty):
                    print(f'({index}): f{np.round(f, 5)}_d{d}', end=' ')
                    task(dcam, alp, f, led, d)



if __name__ == '__main__':
    main()